# CRS Data Loader v2

Simple 6-phase workflow for loading CRS master data:
1. **Tenant & Branding** - Organization setup and UI customization
2. **Boundaries** - Administrative hierarchy (State → District → Block)
3. **Common Masters** - Departments, designations, complaint types
4. **Employees** - Staff accounts with roles and assignments
5. **Localizations** (Optional) - Bulk load translations for new languages
6. **Workflow** - Complaint state machine (APPLY → ASSIGN → RESOLVE)

---

In [1]:
import warnings
import pandas as pd
import requests
from openpyxl import load_workbook
from dataloader.crs_loader import CRSLoader

In [5]:
# Configuration - Edit these values
URL = "http://kong:8000"
USERNAME = "ADMIN"
PASSWORD = "eGov@123"
TENANT_ID = "pg"  # Root tenant for login
TARGET_TENANT = "statea.f"  # Target tenant for data (can be child like "pg.citya")

loader = CRSLoader(URL)
loader.login(username=USERNAME, password=PASSWORD, tenant_id=TENANT_ID)

# Create target tenant if it doesn't exist (also enables PGR & HRMS modules)
loader.create_tenant(TARGET_TENANT, "My Tenant", users=[
      {"username": "ADMIN", "password": "eGov@123", "name": "Admin",
       "roles": ["SUPERUSER", "EMPLOYEE"]}
  ])

loader.login(username="ADMIN", password="eGov@123", tenant_id="statea.f")

✅ Authentication successful!
   User: ADMIN
   Name: System Administrator
   Tenant: pg
   Token: ee1d38e7-eaaa-4a2f-9...
✅ Tenant 'statea.f' already exists
✅ Authentication successful!
   User: ADMIN
   Name: Admin
   Tenant: statea.f
   Token: 0f87263c-be58-4022-8...


True

In [7]:
# Phase 2a - Generate Boundary Template
# Creates hierarchy definition and downloads an Excel template to fill

BOUNDARY_HIERARCHY = "R"  # or "ADMIN" 
BOUNDARY_LEVELS = ["State", "District", "Tehsil"]  # Top to bottom

template_path = loader.load_hierarchy(
    name=BOUNDARY_HIERARCHY,
    levels=BOUNDARY_LEVELS,
    target_tenant=TARGET_TENANT,
    output_dir="upload"
)
print(f"Template saved to: {template_path}")
# Fill the template with your boundary data, then run Phase 2b below


PHASE 2a: BOUNDARY HIERARCHY & TEMPLATE
Tenant: statea.f
Hierarchy: R
Levels: State -> District -> Tehsil

[1/4] Building hierarchy definition...
   Created 3 level definitions

[2/4] Creating hierarchy...

✅ [SUCCESS] Boundary hierarchy created
   Tenant: statea.f
   Hierarchy Type: R
   Levels: 3
   Hierarchy created successfully

[3/4] Generating template...

✅ Template generation initiated
   Task ID: 3b0d1503-4502-4d08-97dd-513293cbebf7
   Status: inprogress

[4/4] Waiting for template...

⏳ Polling for template generation (max 30 attempts)...
   Attempt 1/30: Status = inprogress
   Attempt 2/30: Status = inprogress
   Attempt 3/30: Status = completed

✅ Template generation complete!
   FileStore ID: 77d432a3-1c41-4c54-80c4-7e5ad0579241

📥 Downloading from S3...
✅ Template downloaded: upload/Boundary_Template_statea.f_R.xlsx
📊 File size: 77788 bytes

────────────────────────────────────────
Template downloaded: upload/Boundary_Template_statea.f_R.xlsx
────────────────────────────

In [8]:
# Phase 2b - Load Boundaries from filled template
# Use the template path from Phase 2a, or specify your own filled boundary file

BOUNDARY_FILE = "templates/Boundary_Master.xlsx"  # Or: "upload/boundary_filled_statea_REVENUE.xlsx"

loader.load_boundaries(BOUNDARY_FILE, target_tenant=TARGET_TENANT, hierarchy_type=BOUNDARY_HIERARCHY)


PHASE 2: BOUNDARIES
File: Boundary_Master.xlsx
Hierarchy: R

[1/2] Uploading boundary file...

📤 Uploading file: Boundary_Master.xlsx
✅ File uploaded successfully!
   FileStore ID: 9b056613-5281-4b73-a989-ff324265f4ac
   FileStore ID: 9b056613-5281-4b73-a989-ff324265f4ac

[2/2] Processing boundary data...

📖 Reading boundary data from: templates/Boundary_Master.xlsx
   Found 88 boundary records
   Columns: ['code', 'name', 'boundaryType', 'parentCode']
   Hierarchy: State → District → Tehsil
   Using standard format (code, boundaryType, parentCode)
   Boundary types match hierarchy - no mapping needed
   ✅ Created boundary: PB
   ✅ Created relationship: PB [State] (root)
   ✅ Created boundary: PB_AMR
   ✅ Created relationship: PB_AMR [District] (parent: PB)
   ✅ Created boundary: PB_AMR_AJN
   ✅ Created relationship: PB_AMR_AJN [Tehsil] (parent: PB_AMR)
   ✅ Created boundary: PB_AMR_BBK
   ✅ Created relationship: PB_AMR_BBK [Tehsil] (parent: PB_AMR)
   ✅ Created boundary: PB_AMR_AM1
 

{'status': 'completed',
 'boundaries_created': 88,
 'relationships_created': 67,
 'errors': []}

In [9]:
# Phase 3 - Common Masters
loader.load_common_masters("templates/Common and Complaint Master.xlsx", target_tenant=TARGET_TENANT)


PHASE 3: COMMON MASTERS
File: Common and Complaint Master.xlsx

[1/2] Loading departments & designations...
📥 Fetching departments from MDMS for tenant: statea.f
   ✅ Found 13 department(s)
📥 Fetching designations from MDMS for tenant: statea.f
   ✅ Found 29 designation(s)
   Creating 2 departments...

[UPLOADING] common-masters.Department
   Tenant: statea.f
   Records: 2
   API URL: http://kong:8000/mdms-v2/v2/_create/common-masters.Department
   [OK] [1/2] DEPT_36
   [OK] [2/2] DEPT_37
[SUMMARY] Created: 2
[SUMMARY] Already Exists: 0
[SUMMARY] Failed: 0

📝 Updating Excel file: templates/Common and Complaint Master.xlsx
   Sheet: Department
   ⚠️  Sheet 'Department' not found - skipping status update

[UPLOADING] Localization Messages
   Tenant: statea.f
   Total Messages: 2
   API URL: http://kong:8000/localization/messages/v1/_upsert

   Found 1 locales: en_IN
   📤 Locale: en_IN - Uploading 2 messages in batches of 500...
      ✅ Batch 1/1: 2 messages uploaded
[SUMMARY] Created: 2

{'departments': {'created': 2, 'exists': 0, 'failed': 0, 'errors': []},
 'designations': {'created': 2, 'exists': 0, 'failed': 0, 'errors': []},
 'complaint_types': {'created': 1, 'exists': 0, 'failed': 0, 'errors': []}}

In [10]:
# Phase 4 - Employees
loader.load_employees("templates/Employee_Master_Dynamic_statea.xlsx", target_tenant=TARGET_TENANT)


PHASE 4: EMPLOYEES
File: Employee_Master_Dynamic_statea.xlsx

[1/2] Reading employee data...
📥 Fetching departments from MDMS for tenant: statea.f
   ✅ Found 2 department(s)
📥 Fetching designations from MDMS for tenant: statea.f
   ✅ Found 2 designation(s)
📥 Fetching roles from MDMS for tenant: statea.f
   ✅ Found 21 role(s) from MDMS
   Found 1 employees

[2/2] Creating employees...

🔐 PRE-CHECK: Validating Roles in MDMS

🔍 Checking roles in MDMS for tenant: statea.f
   ✅ Found 21 existing roles in MDMS
   ✅ All 20 required roles already exist in MDMS

[UPLOADING] HRMS Employees
   Tenant: statea.f
   Records: 1
   API URL: http://kong:8000/egov-hrms/employees/_create
   [EXISTS] [1/1] SAMPLE_EMPLOYEE (HTTP 400)
[SUMMARY] Created: 0
[SUMMARY] Password Updated: 0
[SUMMARY] Already Exists: 1
[SUMMARY] Failed: 0

📝 Updating Excel file: templates/Employee_Master_Dynamic_statea.xlsx
   Sheet: Employee Master
   ✅ Status columns updated successfully!
   📊 Updated 1 rows

──────────────────

{'created': 0, 'exists': 1, 'failed': 0, 'errors': [], 'password_updated': 0}

---

## Phase 5: Localizations (Optional)

Load bulk localization messages for a new language. Use this when you need to add Hindi, Tamil, Punjabi, or other regional languages.

**Prerequisites:**
- You need a localization Excel file with a `Localization` sheet
- Required columns: `Code`, `Message`, `Locale` (e.g., `hi_IN`, `pa_IN`)
- Optional columns: `Module`

**To add a new language:**
1. Get the localization template from `templates/localization.xlsx`
2. Fill in translations for the new locale
3. Run the cell below with `language_label` and `locale_code` to enable the language in the UI dropdown

In [11]:
# Phase 5 - Localizations (Optional)
# For bulk translations - upload localization Excel with translated messages

# Option A: Just upload messages (no new language in dropdown)
loader.load_localizations("templates/localization.xlsx", target_tenant=TARGET_TENANT)

# Option B: Upload messages AND enable new language in UI dropdown
# Uncomment and edit the values below:

# loader.load_localizations(
#     "templates/localization.xlsx",
#     target_tenant=TARGET_TENANT,
#     language_label="ਪੰਜਾਬੀ",   # Display name in UI (e.g., "Hindi", "ਪੰਜਾਬੀ", "తెలుగు")
#     locale_code="pa_IN"         # Locale code (e.g., "hi_IN", "pa_IN", "te_IN")
# )


PHASE 5: LOCALIZATIONS
File: localization.xlsx

[1/2] Reading localization data...
   Found 45143 messages
   - en_IN: 45143 messages

[2/2] Uploading localization messages...

[UPLOADING] Localization Messages
   Tenant: statea.f
   Total Messages: 45143
   API URL: http://kong:8000/localization/messages/v1/_upsert

   Found 1 locales: en_IN
   📤 Locale: en_IN - Uploading 45143 messages in batches of 500...
      ⚠️ Batch 1/91: 500 messages already exist
      ⚠️ Batch 2/91: 500 messages already exist
      ⚠️ Batch 3/91: 500 messages already exist
      ⚠️ Batch 4/91: 500 messages already exist
      ⚠️ Batch 5/91: 500 messages already exist
      ⚠️ Batch 6/91: 500 messages already exist
      ⚠️ Batch 7/91: 500 messages already exist
      ⚠️ Batch 8/91: 500 messages already exist
      ⚠️ Batch 9/91: 500 messages already exist
      ⚠️ Batch 10/91: 500 messages already exist
      ⚠️ Batch 11/91: 500 messages already exist
      ⚠️ Batch 12/91: 500 messages already exist
      ⚠️

{'messages': {'created': 4500,
  'exists': 40643,
  'failed': 0,
  'errors': [],
  'failed_records': []},
 'stateinfo': None}

---

## Phase 6: Workflow Configuration

Configure the PGR complaint workflow state machine from a JSON file. This defines the complaint lifecycle:
- **APPLY** → Citizen/CSR creates complaint
- **PENDINGFORASSIGNMENT** → GRO assigns to field worker  
- **PENDINGATLME** → Field worker resolves or reassigns
- **RESOLVED/REJECTED** → Terminal states with rating option

**Template:** Use `../default-data-handler/src/main/resources/PgrWorkflowConfig.json` as a starting point.

The JSON file should have a `BusinessServices` array. Use `{tenantid}` as a placeholder - it will be replaced with your target tenant.

In [12]:
# Phase 6 - Workflow
# Loads the PGR complaint workflow state machine from JSON file

# Default PGR workflow template (copy and modify as needed)
WORKFLOW_JSON = "templates/PgrWorkflowConfig.json"

loader.load_workflow(WORKFLOW_JSON, target_tenant=TARGET_TENANT)


PHASE 6: WORKFLOW
File: PgrWorkflowConfig.json
Tenant: statea.f
Business Service: PGR

[1/3] Loading workflow config from JSON...
   Loaded 11 states

[2/3] Checking for existing workflow...
   Found existing workflow: PGR
   States: 11

[3/3] Workflow already configured (same state count)


{'status': 'exists', 'error': None, 'states': 11}

---

## Rollback / Delete Data

Use these cells to undo data loading. Run only what you need to rollback.

In [ ]:
# Rollback Phase 2 - Delete Boundaries
# Deletes all boundary entities and relationships for the tenant
loader.delete_boundaries(TARGET_TENANT)

In [ ]:
# Rollback Phase 3 - Delete Common Masters
# Deletes departments, designations, and complaint types
loader.rollback_common_masters(TARGET_TENANT)

In [ ]:
# Rollback Phase 1 - Delete Tenant Config
# Deletes tenant and branding configuration
loader.rollback_tenant(TARGET_TENANT)

In [ ]:
# Delete specific MDMS schema data (granular control)
loader.delete_mdms("common-masters.Department", TARGET_TENANT)
loader.delete_mdms("common-masters.Designation", TARGET_TENANT)
loader.delete_mdms("RAINMAKER-PGR.ServiceDefs", TARGET_TENANT)

In [ ]:
# FULL RESET - Delete everything (MDMS + Boundaries)
# WARNING: This deletes ALL data for the tenant!
loader.full_reset("REVENUE", TARGET_TENANT)

**Note:** Employees (Phase 4) cannot be deleted via API - they are managed in HRMS and must be deactivated manually if needed.